In [1]:
import numpy as np
import igraph as ig

Data Generation

In [2]:
theta = np.array([[0.75, 0.25, 0.1, 0.9],
                  [0.75, 0.25, 0.8, 0.1],
                  [0.1, 0.75, 0.25, 0.9]])

In [9]:
Sr = [4, 4, 4]
Sc = [3, 3, 3, 3]

In [10]:
def generate_graph_bicluster(Sr, Sc, thetas):
    # Kr = thetas.shape[0]
    # Kc = thetas.shape[1]

    N = np.sum(Sr)

    Hr = []
    c = 0
    for i in Sr:
        for j in range(i):
            Hr += [c]
        c+=1

    Hc = []
    c = 0
    for i in Sc:
        for j in range(i):
            Hc += [c]
        c+=1

    X = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            X[i, j] = np.random.binomial(1, thetas[Hr[i], Hc[j]])

    return X


In [11]:
X = generate_graph_bicluster(Sr, Sc, theta)

In [12]:
def irm_bicluster():
    
    return Z

array([[0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
       [1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.],
       [0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0.]])

In [22]:
N = len(X)
zr = np.ones([N, 1])
zc = np.ones([N, 1])
Zr = []
Zc = []

In [17]:
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

X_ = X[np.ix_(nn,nn)]

In [29]:
# for t in range(T):
#     for n in range(N):
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

X_ = X[np.ix_(nn,nn)]

Kr = len(zr[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

# m = n. of nodes in each component 
m = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)

# M = max links from other clusts
M = np.tile(m, (Kr, 1))

In [34]:
M = np.tile(m, (Kr, 1))


# M1 = n. of links between components without current node
M1 = zr[nn].T @ X_ @ zr[nn]

# M0 = n. of non-links between components without current node
M0 = m.T@m - np.diag((m*(m+1) / 2).flatten()) - M1 

# r = n. of links from current node to components
r = zr[nn].T @ X[nn, n]
R = np.tile(r, (Kr, 1))

In [39]:
M0

array([[4.]])

In [40]:
X_

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
       [1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.],
       [1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.],
       [1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0.]])

In [35]:
M1

array([[51.]])

In [ ]:
# for t in range(T):
#     for n in range(N):
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

X_ = X[np.ix_(nn,nn)]

Kr = len(zr[0])
Kc = len(zc[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

In [23]:
Kr = len(zr[0])
Kc = len(zc[0])

In [25]:
Kc

1